In [ ]:
name = '2019-05-30-cartopy-basemap'
title = 'Pleasing basemaps with cartopy'
tags = 'matplotlib, cartopy, dataviz, maps'
author = 'Callum Rollo'

# Pleasing basemaps with cartopy

To make a pretty, publication grade basemap for your study area look no further than [cartopy.](https://scitools.org.uk/cartopy/docs/latest/)

In this tutorial we will walk through generating a basemap with:
- Bathymetry/topography
- Coastline
- Scatter data
- Locatinon labels
- Inset map
- Legend

This code can be generalised to any region you wish to map

First we import some modules for manipulating and plotting data


In [ ]:
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Then we import cartopy itself

In [ ]:
import cartopy

In addition, we import cartopy's **coordinate reference system** submodule:

In [ ]:
import cartopy.crs as ccrs

A few other modules and functions which we will use later to add cool stuff to our plots. Also updating font sizes for improved readability

In [ ]:
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.io import shapereader

plt.rcParams.update({'font.size': 20})
SMALL_SIZE = 22
MEDIUM_SIZE = 22
LARGE_SIZE = 26
plt.rc('font', size=SMALL_SIZE)
plt.rc('xtick', labelsize=SMALL_SIZE)
plt.rc('ytick', labelsize=SMALL_SIZE)
plt.rc('axes', titlesize=SMALL_SIZE)
plt.rc('legend', fontsize=SMALL_SIZE)

### Note on bathymetry data
To save space and time I have subset the bathymetry plotted in this example. If you wish to map a different area you will need to download the GEBCO topography data found [here](https://www.gebco.net/data_and_products/gridded_bathymetry_data) then uncomment and run the code block below. I suggest doing this once only as it takes some time! After running this your bathymetry selection will replace the example bathy in the file bathy.pkl

In [ ]:
# from netCDF4 import Dataset
# # A function to find the nearest item to a given point. Very handy with coordinates
# def argnearest(items,pivot):
#     near_item=min(items, key=lambda x: abs(x - pivot))
#     for i in range(len(items)): 
#         if items[i]==near_item:
#             return i
# # Enter the coordinates of your desired box here [lon,lat] -180<lon<180
# SW_vertex = [-42,-60]
# NE_vertex = [-23,-50]
#  
# gebco = Dataset("/path/to/your/GEBCO/folder/GEBCO_2014_2D.nc", "r", format="NETCDF4")
# all_lat = gebco['lat'][:]
# all_lon = gebco['lon'][:]
# 
# 
# SW_indices = [argnearest(all_lon,SW_vertex[0]),argnearest(all_lat,SW_vertex[1])]
# NE_indices = [argnearest(all_lon,NE_vertex[0]),argnearest(all_lat,NE_vertex[1])]
# 
# lon_selec = all_lon[SW_indices[0]:NE_indices[0]+1]
# lat_selec = all_lat[SW_indices[1]:NE_indices[1]+1]
# 
# bath_lat_selec = gebco['elevation'][np.logical_and(all_lat>=lat_selec[0],all_lat<=lat_selec[-1])][:]
# bath_selec = bath_lat_selec[:,np.logical_and(all_lon>=lon_selec[0],all_lon<=lon_selec[-1])]
# 
# # To save our bathymetry data
# bathy_data = {'bathy_lon': lon_selec,
#          'bathy_lat': lat_selec,
#          'bathy_h': bath_selec}
# pickle.dump( bathy_data, open( "bathy.pkl", "wb" ) )


After you've run the code block above once, or you are using the example data, simply load the stored bathymetry data

In [ ]:
bathy_data = pickle.load( open( "bathy.pkl", "rb" ) )
bathy_lon,bathy_lat,bathy_h = map(bathy_data.get, ('bathy_lon', 'bathy_lat','bathy_h'))

We're just interested in bathy here, so set any height values greater than 0 to to 0 and set contour levels to plot later

In [ ]:
bathy_h[bathy_h>0] = 0
bathy_conts = np.arange(-9000,500,500)

Here we load some scatter data from a two column csv for plotting later

In [ ]:
# Load some scatter data of smaple locations near South Georgia
data = pd.read_csv('scatter_coords.csv')
lons = data.Longitude.values
lats = data.Latitude.values

# Subset of sampling locations
sample_lon = lons[[0,2,7]]
sample_lat = lats[[0,2,7]]


Now to make the map itself. First we define our coordinate system. Here we are using a Plate Carrée projection, which is one of *equidistant cylindrical projections*.

A full list of Cartopy projections is available at http://scitools.org.uk/cartopy/docs/latest/crs/projections.html.

Then we create figure and axes instances and set the plotting extent in degrees [West, East, South, North]

In [ ]:
coord=ccrs.PlateCarree()
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection=coord)
ax.set_extent([-42, -23, -60, -50], crs=coord)

Now we contour the bathymetry data

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection=coord)
ax.set_extent([-42, -23, -60, -50], crs=coord)
bathy=ax.contourf(bathy_lon,bathy_lat,bathy_h,bathy_conts,transform=coord,cmap='Blues_r')

A good start. To make it more map like we add gridlines, formatted labels and a colorbar

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection=coord)
ax.set_extent([-42, -23, -60, -50], crs=coord)
bathy=ax.contourf(bathy_lon,bathy_lat,bathy_h,bathy_conts,transform=coord,cmap='Blues_r')

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='k', alpha=0.5, linestyle='--')
gl.xlabels_top = False
gl.ylabels_right = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.ylines = True
gl.xlines = True
fig.colorbar(bathy,ax=ax,orientation='horizontal',label='Bathymetry (m)',shrink=0.7,pad=0.08,aspect=40);

Now to add a few more features. First coastlines from cartopy's natural features toolbox. Then scatters of the samples we imported earlier

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection=coord)
ax.set_extent([-42, -23, -60, -50], crs=coord)
bathy=ax.contourf(bathy_lon,bathy_lat,bathy_h,bathy_conts,transform=coord,cmap='Blues_r')

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='k', alpha=0.5, linestyle='--')
gl.xlabels_top = False
gl.ylabels_right = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.ylines = True
gl.xlines = True
fig.colorbar(bathy,ax=ax,orientation='horizontal',label='Bathymetry (m)',shrink=0.7,pad=0.08,aspect=40)


feature = cartopy.feature.NaturalEarthFeature(name='coastline',
                                              category='physical',
                                              scale='50m',
                                              edgecolor='0.5',
                                              facecolor='0.8')
ax.add_feature(feature)
ax.scatter(lons,lats,zorder=5,color='red',label='Samples collected')
ax.scatter(sample_lon,sample_lat,zorder=10,color='k',marker='D',s=50,label='Samples sequenced');

To finish off the map we add a legend for the scatter plot, an inset map showing the area at a larger scale and some text identifying the islands

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection=coord)
ax.set_extent([-42, -23, -60, -50], crs=coord)
bathy=ax.contourf(bathy_lon,bathy_lat,bathy_h,bathy_conts,transform=coord,cmap='Blues_r')

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='k', alpha=0.5, linestyle='--')
gl.xlabels_top = False
gl.ylabels_right = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.ylines = True
gl.xlines = True
fig.colorbar(bathy,ax=ax,orientation='horizontal',label='Bathymetry (m)',shrink=0.7,pad=0.08,aspect=40)
ax.add_feature(feature)
ax.scatter(lons,lats,zorder=5,color='red',label='Samples collected')
ax.scatter(sample_lon,sample_lat,zorder=10,color='k',marker='D',s=50,label='Samples sequenced')
fig .legend( bbox_to_anchor=(0.12,0.2),loc='lower left')

tr2=ccrs.Stereographic(central_latitude=-55,central_longitude=-35)
sub_ax = plt.axes([0.63, 0.65, 0.2, 0.2], projection=ccrs.Stereographic(central_latitude=-55,central_longitude=-35))
sub_ax.set_extent([-70, -15, -75, 10])
x_co = [-42,-42,-23,-23,-42]
y_co = [-60,-50,-50,-60,-60]
sub_ax.add_feature(feature)
sub_ax.plot(x_co,y_co,transform=coord,zorder=10,color='red')

ax.text(-38.5,-54.9,'South\nGeorgia',fontsize=14)
ax.text(-26.8,-58.2,'South\nSandwich\nIslands',fontsize=14);

In [ ]:
HTML(html)